Predizendo Preço de games na Steam

In [208]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error
import numpy as np


In [209]:
df = pd.read_json('steamdb.json')
df.head(10)

In [210]:
df.info()

A base de dados têm bastante indices nulos e colunas que não nos dão muita informação como URL do game.
Precisamos retirar itens chave e manter itens que possam ter alguma relevância mas não tenham tantos NaN.

In [211]:

indices_to_drop = [1,2,4,5,6,7,8,9,11,14,15,16,17,18,19,24,26,27,28,30,32,33,36,37,38,39,40,41,42,43,44,45]

# Obter os nomes das colunas correspondentes aos índices
col_names_to_drop = df.columns[indices_to_drop]

# Remover as colunas
df = df.drop(columns=col_names_to_drop)

In [212]:
df.isna().sum()

In [213]:
df.dropna().shape

In [214]:
df.info()

In [215]:
df.nunique()

Podemos categorizar a dificuldade e tentar expandir os multivalorados 'categories' e 'tags'

In [216]:
df['gfq_difficulty'] = df['gfq_difficulty'].astype('category')

df['name']=df['name'].astype('string')

In [217]:
df['genres'].unique()

In [218]:
df['categories'] = df['categories'].apply(lambda x: x.split(',') if x is not None else [])
df['genres'] = df['genres'].apply(lambda x: x.split(',') if x is not None else [])
df['tags'] = df['tags'].apply(lambda x: x.split(',') if x is not None else [])
df.head()

In [219]:
colunas_para_verificar = ['tags', 'categories', 'genres']

# Contando valores únicos para cada coluna
for coluna in colunas_para_verificar:
    # Achatar a lista de listas
    itens_achatados = [item for sublist in df[coluna].dropna() for item in sublist]

    # Obter os valores únicos
    itens_unicos = set(itens_achatados)

    # Contar e exibir o número de itens únicos
    print(f"Número de itens únicos em '{coluna}': {len(itens_unicos)}")

Temos portanto um grande número de generos e categorias e ainda maior de tags. 
Expandir esses indices pode levar a um número muito extenso de colunas e valores para administrar, isso seria adequado para outros tipos de modelo, diferentes dos que estamos trabalhando na matéria. portanto vamos trabalhar apenas com as outras colunas.

In [220]:
# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
combined_labels = df.apply(lambda x: set(x['categories'] + x['genres'] + x['tags']), axis=1)

# Instanciar o MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
combined_encoded = pd.DataFrame(mlb.fit_transform(combined_labels), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original
df_encoded = pd.concat([df, combined_encoded], axis=1)

df_encoded.shape

In [221]:
df_encoded = df_encoded.dropna()
df_encoded = df_encoded.drop(columns='full_price')
df_encoded_1 = df_encoded.copy()
df_encoded_1['Multi'] = df_encoded_1['Multi-player'] | df_encoded_1['Multiplayer']
df_encoded_2 = df_encoded_1.drop(columns=['Multiplayer', 'Multi-player'])
#df_encoded original com 2 colunas multiplayer
#df_encoded_1 com 3 colunas multiplayer, adicionada uma coluna unificada com operador or
#df_encoded_2 com 1 coluna multiplayer, deixando apenas a unificada

In [222]:
# Calcular a matriz de correlação
corr_matrix = df_encoded.corr()
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']       # Variável dependente
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()

corr_matrix = df_encoded_1.corr()
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded_1[cols_to_keep]  # Variáveis independentes
y = df_encoded_1['current_price']       # Variável dependente
X_sm_1 = sm.add_constant(X)
model_sm_1 = sm.OLS(y, X_sm_1).fit()

corr_matrix = df_encoded_2.corr()
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded_2[cols_to_keep]  # Variáveis independentes
y = df_encoded_2['current_price']       # Variável dependente
X_sm_2 = sm.add_constant(X)
model_sm_2 = sm.OLS(y, X_sm_2).fit()

# Exibir o sumário do modelo
print(model_sm.summary())
print(model_sm_1.summary())
print(model_sm_2.summary())

In [177]:
corr_matrix = df_encoded.corr()

# Filtrar colunas com correlação acima de um limiar (0.1 neste caso)
# Verificar primeiro se as colunas estão presentes no DataFrame
cols_to_keep = corr_matrix.columns[corr_matrix.abs()['current_price'] > 0.2]

# Criar uma nova matriz de correlação apenas com as colunas filtradas
filtered_corr_matrix = df_encoded[cols_to_keep].corr()

# Usar seaborn para criar um heatmap da matriz de correlação filtrada
plt.figure(figsize=(20, 20))
sns.heatmap(filtered_corr_matrix, annot=True, fmt=".2f", cmap='bwr', center= 0.05)

# Mostrar o gráfico
plt.show()

In [178]:
# Calcular a matriz de correlação
corr_matrix = df_encoded.corr()

# Identificar colunas com correlação maior que 0.05 com a coluna 'score'
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista

# Preparar os dados
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']       # Variável dependente

# Dividir o conjunto de dados em treino e teste
X_sm = sm.add_constant(X)

# Construir e treinar o modelo
model_sm = sm.OLS(y, X_sm).fit()

# Exibir o sumário do modelo
print(model_sm.summary())

In [169]:
print(df_encoded['Multi-player'].sum())
print(df_encoded['Multiplayer'].sum())

In [170]:
df_encoded['Multi'] = df_encoded['Multi-player'] | df_encoded['Multiplayer']

In [171]:
corr_matrix = df_encoded.corr()

# Identificar colunas com correlação maior que 0.05 com a coluna 'score'
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista

# Preparar os dados
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']       # Variável dependente

# Dividir o conjunto de dados em treino e teste
X_sm = sm.add_constant(X)

# Construir e treinar o modelo
model_sm = sm.OLS(y, X_sm).fit()

# Exibir o sumário do modelo
print(model_sm.summary())

In [70]:
df = df.drop(columns=['tags','categories', 'genres'])
df = df.dropna()
df.info()

In [72]:
corr_matrix = df.corr()

# Usar seaborn para criar um heatmap (mapa de calor) da matriz de correlação
plt.figure(figsize=(10, 10))  # Ajustar o tamanho da figura
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')

# Mostrar o gráfico
plt.show()

In [ ]:


# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
tags_encoded = pd.DataFrame(mlb.fit_transform(df['tags']), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original (se necessário)
df_encoded = pd.concat([df, tags_encoded], axis=1)
df_encoded.shape

In [ ]:
df['categories'] = df['categories'].apply(lambda x: x.split(',') if x is not None else [])

# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
tags_encoded = pd.DataFrame(mlb.fit_transform(df['categories']), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original (se necessário)
df_encoded = pd.concat([df, tags_encoded], axis=1)
df_encoded.shape

O tamanho do

In [15]:
df_clean = df.dropna()
df_clean.info()

#multivalorado
for i in [15,18,19,20,22]:

#categorizar
for i in [21, 25]:

In [17]:


df['categories'] = df['categories'].apply(lambda x: x.split(',') if x is not None else [])

# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
tags_encoded = pd.DataFrame(mlb.fit_transform(df['categories']), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original (se necessário)
df_encoded = pd.concat([df, tags_encoded], axis=1)
df_encoded.shape

In [19]:
df_encoded.columns